## 라이브러리

In [1]:
# !pip install gmplot

In [2]:
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import pandas as pd
import webbrowser
import base64
import gmplot
import folium
import branca
import json
import html
import re
import os

from io import BytesIO
from IPython.display import IFrame
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from folium.plugins import MarkerCluster

## 데이터 확인

In [2]:
df_per = pd.read_csv("project4/독거노인가구비율.csv", encoding="euc-kr")

In [3]:
df = pd.read_csv("project4/노인돌봄서비스_위경도.csv", encoding = "EUC-KR")
len(df)

670

In [4]:
df_t = pd.read_excel("project4/응급안전안심서비스_통합.xlsx", sheet_name="통합 데이터")

In [5]:
df_e = df[df["응급안전안심서비스"] == "해당"].copy()
len(df_e)

186

In [6]:
df_o = df[(df["응급안전안심서비스"] == "해당") & (df["방문요양서비스"] == "해당")].copy()
len(df_o)

19

In [7]:
df_all = df[(df["응급안전안심서비스"] == "해당") & (df["방문요양서비스"] == "해당") &
(df["재가노인복지서비스"] == "해당") & (df["특화서비스"] == "해당")]
len(df_all)

9

## 위경도 추가

In [102]:
# CSV 파일 불러오기 (EUC-KR 인코딩 사용)
df = pd.read_csv("project4/노인돌봄서비스_위경도_완료.csv", encoding="EUC-KR")

# 정규표현식을 사용하여 괄호 이후의 텍스트 제거하는 함수
def clean_address(address):
    if pd.notna(address):
        address = re.sub(r"\(.*\)", "", address)
        address = re.sub(r"[,/].*", "", address)
        address = address.strip()
    return address

# 새로운 칼럼 '주소2' 생성
df["주소2"] = df["주소"].apply(clean_address)

# 변경된 데이터 확인
df.head()

# 결과를 새로운 CSV 파일로 저장
output_path = "project4/노인돌봄서비스_주소정리완료.csv"
df.to_csv(output_path, encoding="EUC-KR", index=False)

In [103]:
# 파일 경로 설정 (CSV 파일의 경로를 직접 지정)
file_path = "project4/노인돌봄서비스_주소정리완료.csv"

# CSV 파일 불러오기
df_l = pd.read_csv(file_path, encoding="EUC-KR")

# Geocoder 설정
geolocator = Nominatim(user_agent="geo_converter")

# 주소를 위경도로 변환하는 함수
def get_lat_lon(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None

# 위도, 경도 변환 실행
df_l[['위도', '경도']] = df_l['주소'].apply(lambda x: pd.Series(get_lat_lon(x)))

# 변환된 데이터 확인
print(df_l.head())

# 변환된 데이터 저장 (새로운 CSV 파일로 저장)
output_path = "project4/노인돌봄서비스_위경도.csv"
df_l.to_csv(output_path, encoding="EUC-KR", index=False)

   연번     시도  시군구          기관명                        주소      기관 대표전화  \
0   1  서울특별시  강남구  강남구노인통합지원센터  서울특별시 강남구 봉은사로24길 70, 2층  02-557-8091   
1   2  서울특별시  강동구  시립강동노인종합복지관     서울특별시 강동구 동남로71길 32-5  02-442-1026   
2   3  서울특별시  강동구   성가정노인종합복지관     서울특별시 강동구 양재대로156길 28  02-481-2218   
3   4  서울특별시  강동구    성내종합사회복지관       서울특별시 강동구 성안로13길 56  02-478-2555   
4   5  서울특별시  강북구  시립강북노인종합복지관  서울특별시 강북구 삼양로92길 40(인수동)  02-999-9179   

                          위수탁기간    시설유형구분 특화서비스 응급안전안심서비스 방문요양서비스 재가노인복지서비스  \
0  2022-03-01 ~ 2025-02-28 (3년)  재가노인복지시설    해당      해당없음    해당없음      해당없음   
1  2023-01-01 ~ 2024-12-31 (2년)  노인여가복지시설  해당없음      해당없음    해당없음      해당없음   
2  2023-01-01 ~ 2024-12-31 (2년)  노인여가복지시설  해당없음      해당없음    해당없음      해당없음   
3  2023-01-01 ~ 2024-12-31 (2년)     사회복지관  해당없음      해당없음    해당없음      해당없음   
4  2023-01-01 ~ 2025-12-31 (3년)  노인여가복지시설    해당      해당없음    해당없음      해당없음   

  사회서비스원 소속 사회서비스형 노인일자리 파견 이용         위도          경도                    주소2  
0      

## 지도 시각화

#### 서비스 대상자, 기관 수

In [8]:
# 지도 중심 설정 (대한민국 중심)
m = folium.Map(location=[36, 128], zoom_start=7, tiles="cartodb positron")

# "서비스 이용자"을 숫자로 변환 (천 단위 구분 기호 제거 후 변환)
df_per["서비스 이용자"] = df_per["서비스 이용자"].astype(str).str.replace(",", "").astype(float)

# 최소 및 최대값 다시 가져오기
min_target = df_per["서비스 이용자"].min()
max_target = df_per["서비스 이용자"].max()

# 색상 스케일 설정
colormap = branca.colormap.LinearColormap(
    colors=["#FEE5D9", "#FC9272", "#DE2D26"],  # 밝은색 → 진한색 (적은 대상자 → 많은 대상자)
    vmin=min_target,
    vmax=max_target
)

# GeoJSON 파일 로드
with open("project4/TL_SCCO_CTPRVN.json", encoding="utf-8") as f:
    geojson_data = json.load(f)

# 지역별 서비스 이용자 수 매핑
region_target_map = df_per.set_index("행정구역")["서비스 이용자"].to_dict()

# GeoJSON 속성에 '독거 노인 가구 비율' 데이터 추가
for feature in geojson_data["features"]:
    region_name = feature["properties"]["CTP_KOR_NM"]
    service_count = region_target_map.get(region_name, "N/A")
    
    if service_count != "N/A":
        feature["properties"]["서비스 이용자"] = f"{int(service_count):,} 명"  # 정수로 변환
    else:
        feature["properties"]["서비스 이용자"] = "N/A"

# 지도에 Choropleth(색상 맵) 추가
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    name="Choropleth", 
    data=df_per,
    columns=["행정구역", "서비스 이용자"],
    key_on="feature.properties.CTP_KOR_NM",
    fill_color="YlOrRd",  # 빨강 계열 색상 적용
    fill_opacity=0.9,  # 투명도 설정
    line_opacity=0.4,  # 경계선 투명도
    legend_name="서비스 이용자(명)",
    highlight=True
).add_to(m)

# 지역 이름 + 비율을 툴팁으로 표시
folium.GeoJsonTooltip(
    fields=["CTP_KOR_NM", "서비스 이용자"],
    aliases=["지역명:", "서비스 이용자:"],
    localize=True,
    sticky=True,
    labels=True,
    style="font-size: 12px; background-color: white; color: black;"
).add_to(choropleth.geojson)

m.save("project4/서비스이용자.html")

IFrame("project4/서비스이용자.html", width=800, height=600)

In [9]:
# "서비스 이용자"을 숫자로 변환 (천 단위 구분 기호 제거 후 변환)
df_per["서비스 이용자"] = df_per["서비스 이용자"].astype(str).str.replace(",", "").astype(float)

# GeoJSON 파일 로드
with open("project4/TL_SCCO_CTPRVN.json", encoding="utf-8") as f:
    geojson_data = json.load(f)

# 지도 초기화 (대한민국 중심)
m = folium.Map(location=[36, 128], zoom_start=7, tiles="cartodb positron")

# 색상 스케일 설정 (서비스 이용자 기준)
min_target = df_per["서비스 이용자"].min()
max_target = df_per["서비스 이용자"].max()
colormap = branca.colormap.LinearColormap(
    colors=["#FEE5D9", "#FC9272", "#DE2D26"],
    vmin=min_target,
    vmax=max_target,
    caption="서비스 이용자"
)

# GeoJSON 속성에 '서비스 이용자' 데이터 추가
region_target_map = df_per.set_index("행정구역")["서비스 이용자"].to_dict()
for feature in geojson_data["features"]:
    region_name = feature["properties"]["CTP_KOR_NM"]
    service_count = region_target_map.get(region_name, "N/A")
    
    if service_count != "N/A":
        feature["properties"]["서비스 이용자"] = f"{int(service_count):,} 명"  # 정수로 변환
    else:
        feature["properties"]["서비스 이용자"] = "N/A"

# Choropleth(색상 맵) 추가 (초기 fill_opacity=0 설정)
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    name="Choropleth",
    data=df_per,
    columns=["행정구역", "서비스 이용자"],
    key_on="feature.properties.CTP_KOR_NM",
    fill_color="YlOrRd",
    fill_opacity=0,
    line_opacity=0.4,
    legend_name="서비스 이용자(명)",
    highlight=True
).add_to(m)

# 지역명 + 서비스 이용자 수 툴팁 추가
folium.GeoJsonTooltip(
    fields=["CTP_KOR_NM", "서비스 이용자"],
    aliases=["지역명:", "서비스 이용자:"],
    localize=True,
    sticky=True,
    labels=True,
    style="font-size: 12px; background-color: white; color: black;"
).add_to(choropleth.geojson)

# 시도별 개별 FeatureGroup을 사용하여 클러스터링 (기본 OFF)
sido_groups = {sido: folium.FeatureGroup(name=sido, show=False).add_to(m) for sido in df["시도"].unique()}

# 노인돌봄서비스 기관 마커 추가 (시도별 FeatureGroup 사용)
for _, row in df.iterrows():
    lat, lon, name, sido = row["위도"], row["경도"], row["기관명"], row["시도"]
    if not pd.isna(lat) and not pd.isna(lon):
        folium.Marker(
            location=[lat, lon],
            popup=folium.Popup(name, parse_html=True),
            tooltip=name,
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(sido_groups[sido])  # ✅ 특정 시도의 그룹에 추가

# LayerControl 추가 (시도별 마커 & Choropleth 기본 OFF)
folium.LayerControl(collapsed=False).add_to(m)

# Choropleth 기본 OFF + 체크 시 fill_opacity 변경 (JavaScript 수정)
script = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    setTimeout(function() {
        var checkboxes = document.querySelectorAll("input.leaflet-control-layers-selector");
        checkboxes.forEach(function(checkbox) {
            if (checkbox.nextSibling && checkbox.nextSibling.textContent.trim() === "Choropleth") {
                checkbox.checked = false;  // Choropleth 기본 OFF
                checkbox.dispatchEvent(new Event("change"));  // 업데이트 트리거
            }
        });

        // Choropleth 투명도 조절하여 숨김
        var choroplethLayer = document.querySelector(".leaflet-overlay-pane");
        if (choroplethLayer) {
            choroplethLayer.style.opacity = "0";  // Choropleth 기본적으로 투명하게 설정
        }

        // 체크박스를 다시 클릭하면 Choropleth가 보이도록 설정 (fill_opacity 조정)
        document.querySelectorAll("input.leaflet-control-layers-selector").forEach(function(checkbox) {
            checkbox.addEventListener("change", function() {
                if (checkbox.nextSibling && checkbox.nextSibling.textContent.trim() === "Choropleth") {
                    var choroplethLayer = document.querySelector(".leaflet-overlay-pane");
                    if (checkbox.checked) {
                        choroplethLayer.style.opacity = "1";  // 다시 보이게
                        choroplethLayer.querySelectorAll("path").forEach(function(path) {
                            path.style.fillOpacity = "0.9";  // ✅ Choropleth 색상 복구
                        });
                    } else {
                        choroplethLayer.style.opacity = "0";  // 숨김
                        choroplethLayer.querySelectorAll("path").forEach(function(path) {
                            path.style.fillOpacity = "0";  // ✅ Choropleth 색상 숨김
                        });
                    }
                }
            });
        });
    }, 100);
});
</script>
"""
m.get_root().html.add_child(folium.Element(script))

# 지도 저장
m.save("project4/노인돌봄서비스기관.html")

# Jupyter Notebook에서 지도 보기
IFrame("project4/노인돌봄서비스기관.html", width=800, height=600)

In [10]:
# "서비스 이용자"을 숫자로 변환 (천 단위 구분 기호 제거 후 변환)
df_per["서비스 이용자"] = df_per["서비스 이용자"].astype(str).str.replace(",", "").astype(float)

# GeoJSON 파일 로드
with open("project4/TL_SCCO_CTPRVN.json", encoding="utf-8") as f:
    geojson_data = json.load(f)

# 지도 초기화 (대한민국 중심)
m = folium.Map(location=[36, 128], zoom_start=7, tiles="cartodb positron")

# 색상 스케일 설정 (서비스 이용자 기준)
min_target = df_per["서비스 이용자"].min()
max_target = df_per["서비스 이용자"].max()
colormap = branca.colormap.LinearColormap(
    colors=["#FEE5D9", "#FC9272", "#DE2D26"],
    vmin=min_target,
    vmax=max_target,
    caption="서비스 대상자"
)

# GeoJSON 속성에 '서비스 이용자' 데이터 추가
region_target_map = df_per.set_index("행정구역")["서비스 이용자"].to_dict()
for feature in geojson_data["features"]:
    region_name = feature["properties"]["CTP_KOR_NM"]
    service_count = region_target_map.get(region_name, "N/A")
    
    if service_count != "N/A":
        feature["properties"]["서비스 이용자"] = f"{int(service_count):,} 명"  # 정수로 변환
    else:
        feature["properties"]["서비스 이용자"] = "N/A"

# Choropleth(색상 맵) 추가 (초기 fill_opacity=0 설정)
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    name="Choropleth",
    data=df_per,
    columns=["행정구역", "서비스 이용자"],
    key_on="feature.properties.CTP_KOR_NM",
    fill_color="YlOrRd",
    fill_opacity=0,
    line_opacity=0.4,
    legend_name="서비스 이용자(명)",
    highlight=True
).add_to(m)

# 지역명 + 서비스 이용자 수 툴팁 추가
folium.GeoJsonTooltip(
    fields=["CTP_KOR_NM", "서비스 이용자"],
    aliases=["지역명:", "서비스 이용자:"],
    localize=True,
    sticky=True,
    labels=True,
    style="font-size: 12px; background-color: white; color: black;"
).add_to(choropleth.geojson)

# 시도별 개별 FeatureGroup을 사용하여 클러스터링 (기본 OFF)
sido_groups = {sido: folium.FeatureGroup(name=sido, show=False).add_to(m) for sido in df["시도"].unique()}

# 응급안전안심서비스 마커 추가 (시도별 FeatureGroup 사용)
for _, row in df_e.iterrows():
    lat, lon, name, sido = row["위도"], row["경도"], row["기관명"], row["시도"]
    if not pd.isna(lat) and not pd.isna(lon):
        folium.Marker(
            location=[lat, lon],
            popup=folium.Popup(name, parse_html=True),
            tooltip=name,
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(sido_groups[sido])  # ✅ 특정 시도의 그룹에 추가

# LayerControl 추가 (시도별 마커 & Choropleth 기본 OFF)
folium.LayerControl(collapsed=False).add_to(m)

# Choropleth 기본 OFF + 체크 시 fill_opacity 변경 (JavaScript 수정)
script = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    setTimeout(function() {
        var checkboxes = document.querySelectorAll("input.leaflet-control-layers-selector");
        checkboxes.forEach(function(checkbox) {
            if (checkbox.nextSibling && checkbox.nextSibling.textContent.trim() === "Choropleth") {
                checkbox.checked = false;  // Choropleth 기본 OFF
                checkbox.dispatchEvent(new Event("change"));  // 업데이트 트리거
            }
        });

        // Choropleth 투명도 조절하여 숨김
        var choroplethLayer = document.querySelector(".leaflet-overlay-pane");
        if (choroplethLayer) {
            choroplethLayer.style.opacity = "0";  // Choropleth 기본적으로 투명하게 설정
        }

        // 체크박스를 다시 클릭하면 Choropleth가 보이도록 설정 (fill_opacity 조정)
        document.querySelectorAll("input.leaflet-control-layers-selector").forEach(function(checkbox) {
            checkbox.addEventListener("change", function() {
                if (checkbox.nextSibling && checkbox.nextSibling.textContent.trim() === "Choropleth") {
                    var choroplethLayer = document.querySelector(".leaflet-overlay-pane");
                    if (checkbox.checked) {
                        choroplethLayer.style.opacity = "1";  // 다시 보이게
                        choroplethLayer.querySelectorAll("path").forEach(function(path) {
                            path.style.fillOpacity = "0.9";  // ✅ Choropleth 색상 복구
                        });
                    } else {
                        choroplethLayer.style.opacity = "0";  // 숨김
                        choroplethLayer.querySelectorAll("path").forEach(function(path) {
                            path.style.fillOpacity = "0";  // ✅ Choropleth 색상 숨김
                        });
                    }
                }
            });
        });
    }, 100);
});
</script>
"""
m.get_root().html.add_child(folium.Element(script))

# 지도 저장
m.save("project4/응급안전안심서비스.html")

# Jupyter Notebook에서 지도 보기
IFrame("project4/응급안전안심서비스.html", width=800, height=600)

In [11]:
# "서비스 이용자"을 숫자로 변환 (천 단위 구분 기호 제거 후 변환)
df_per["서비스 이용자"] = df_per["서비스 이용자"].astype(str).str.replace(",", "").astype(float)

# GeoJSON 파일 로드
with open("project4/TL_SCCO_CTPRVN.json", encoding="utf-8") as f:
    geojson_data = json.load(f)

# 지도 초기화 (대한민국 중심)
m = folium.Map(location=[36, 128], zoom_start=7, tiles="cartodb positron")

# 색상 스케일 설정 (서비스 이용자 기준)
min_target = df_per["서비스 이용자"].min()
max_target = df_per["서비스 이용자"].max()
colormap = branca.colormap.LinearColormap(
    colors=["#FEE5D9", "#FC9272", "#DE2D26"],
    vmin=min_target,
    vmax=max_target,
    caption="서비스 이용자"
)

# GeoJSON 속성에 '서비스 이용자' 데이터 추가
region_target_map = df_per.set_index("행정구역")["서비스 이용자"].to_dict()
for feature in geojson_data["features"]:
    region_name = feature["properties"]["CTP_KOR_NM"]
    service_count = region_target_map.get(region_name, "N/A")
    
    if service_count != "N/A":
        feature["properties"]["서비스 이용자"] = f"{int(service_count):,} 명"  # 정수로 변환
    else:
        feature["properties"]["서비스 이용자"] = "N/A"

# Choropleth(색상 맵) 추가 (초기 fill_opacity=0 설정)
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    name="Choropleth",
    data=df_per,
    columns=["행정구역", "서비스 이용자"],
    key_on="feature.properties.CTP_KOR_NM",
    fill_color="YlOrRd",
    fill_opacity=0,
    line_opacity=0.4,
    legend_name="서비스 이용자(명)",
    highlight=True
).add_to(m)

# 지역명 + 서비스 이용자 수 툴팁 추가
folium.GeoJsonTooltip(
    fields=["CTP_KOR_NM", "서비스 이용자"],
    aliases=["지역명:", "서비스 이용자:"],
    localize=True,
    sticky=True,
    labels=True,
    style="font-size: 12px; background-color: white; color: black;"
).add_to(choropleth.geojson)

# 시도별 개별 FeatureGroup을 사용하여 클러스터링 (기본 OFF)
sido_groups = {sido: folium.FeatureGroup(name=sido, show=False).add_to(m) for sido in df["시도"].unique()}

# 응급안전안심, 방문요양서비스 기관 마커 추가 (시도별 FeatureGroup 사용)
for _, row in df_o.iterrows():
    lat, lon, name, sido = row["위도"], row["경도"], row["기관명"], row["시도"]
    if not pd.isna(lat) and not pd.isna(lon):
        folium.Marker(
            location=[lat, lon],
            popup=folium.Popup(name, parse_html=True),
            tooltip=name,
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(sido_groups[sido])  # ✅ 특정 시도의 그룹에 추가

# LayerControl 추가 (시도별 마커 & Choropleth 기본 OFF)
folium.LayerControl(collapsed=False).add_to(m)

# Choropleth 기본 OFF + 체크 시 fill_opacity 변경 (JavaScript 수정)
script = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    setTimeout(function() {
        var checkboxes = document.querySelectorAll("input.leaflet-control-layers-selector");
        checkboxes.forEach(function(checkbox) {
            if (checkbox.nextSibling && checkbox.nextSibling.textContent.trim() === "Choropleth") {
                checkbox.checked = false;  // Choropleth 기본 OFF
                checkbox.dispatchEvent(new Event("change"));  // 업데이트 트리거
            }
        });

        // Choropleth 투명도 조절하여 숨김
        var choroplethLayer = document.querySelector(".leaflet-overlay-pane");
        if (choroplethLayer) {
            choroplethLayer.style.opacity = "0";  // Choropleth 기본적으로 투명하게 설정
        }

        // 체크박스를 다시 클릭하면 Choropleth가 보이도록 설정 (fill_opacity 조정)
        document.querySelectorAll("input.leaflet-control-layers-selector").forEach(function(checkbox) {
            checkbox.addEventListener("change", function() {
                if (checkbox.nextSibling && checkbox.nextSibling.textContent.trim() === "Choropleth") {
                    var choroplethLayer = document.querySelector(".leaflet-overlay-pane");
                    if (checkbox.checked) {
                        choroplethLayer.style.opacity = "1";  // 다시 보이게
                        choroplethLayer.querySelectorAll("path").forEach(function(path) {
                            path.style.fillOpacity = "0.9";  // ✅ Choropleth 색상 복구
                        });
                    } else {
                        choroplethLayer.style.opacity = "0";  // 숨김
                        choroplethLayer.querySelectorAll("path").forEach(function(path) {
                            path.style.fillOpacity = "0";  // ✅ Choropleth 색상 숨김
                        });
                    }
                }
            });
        });
    }, 100);
});
</script>
"""
m.get_root().html.add_child(folium.Element(script))

# 지도 저장
m.save("project4/응급안전안심_방문요양서비스.html")

# Jupyter Notebook에서 지도 보기
IFrame("project4/응급안전안심_방문요양서비스.html", width=800, height=600)

In [14]:
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>안심서비스 현황 지도</title>
    <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
    <style>
        .radio-container {{
            position: absolute;
            top: 10px;
            left: 10px;
            background: rgba(255, 255, 255, 0.9);
            padding: 10px;
            border-radius: 8px;
            z-index: 1000;
            font-size: 14px;
            box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.2);
            display: flex;
            flex-direction: column;
            gap: 5px;
        }}
        .map-container {{
            display: none;
            width: 100%;
            height: 100vh;
        }}
    </style>
</head>
<body>
    <div class="radio-container">
        <label><input type="radio" name="mapSelect" value="care" checked> 노인돌봄서비스기관</label>
        <label><input type="radio" name="mapSelect" value="safety"> 응급안전안심서비스</label>
        <label><input type="radio" name="mapSelect" value="safety_care"> 응급안전안심 방문요양서비스</label>
    </div>

    <iframe id="care" class="map-container" src="project4/노인돌봄서비스기관.html" style="display:block;"></iframe>
    <iframe id="safety" class="map-container" src="project4/응급안전안심서비스.html"></iframe>
    <iframe id="safety_care" class="map-container" src="project4/응급안전안심_방문요양서비스.html"></iframe>

    <script>
        $(document).ready(function() {{
            $('input[name="mapSelect"]').change(function() {{
                $('.map-container').hide();
                $('#' + $(this).val()).show();
            }});
        }});
    </script>
</body>
</html>
"""

# HTML 파일 저장
with open("독거노인.html", "w", encoding="utf-8") as f:
    f.write(html_content)

# Jupyter Notebook에서 HTML 파일 실행
webbrowser.open("독거노인.html") 

True

#### 응급관리요원 수

In [12]:
plt.rc("font", family="Malgun Gothic")

# 연도 리스트
years = ["2020년", "2021년", "2022년", "2023년", "2024년"]

# "서비스 대상자(명)"을 숫자로 변환
for year in years:
    df_t[f"{year}_서비스이용자"] = df_t[f"{year}_서비스이용자"].astype(int)
    df_t[f"{year}_응급관리요원"] = df_t[f"{year}_응급관리요원"].astype(int)


# 지역별 차트 생성 (Base64 변환)
region_chart_base64 = {}

# 전체 데이터에서 최댓값 찾기 (Y축 범위 통일)
global_y_max = df_t[[f"{year}_서비스이용자" for year in years]].max().max() + 5000
global_y2_max = df_t[[f"{year}_응급관리요원" for year in years]].max().max() + 10

for _, row in df_t.iterrows():
    region = row["행정구역"]
    service_users = row[[f"{year}_서비스이용자" for year in years]].values
    emergency_staff = row[[f"{year}_응급관리요원" for year in years]].values

    fig, ax1 = plt.subplots(figsize=(5, 3))

    # 서비스 이용자 수 → 막대그래프로 표시 (왼쪽 Y축)
    ax1.bar(years, service_users, color="royalblue", alpha=0.7, label="서비스 이용자 수")
    ax1.set_ylabel("서비스 이용자 수", color="royalblue")
    ax1.set_ylim(0, global_y_max)
    ax1.yaxis.set_major_locator(ticker.MultipleLocator(5000))
    ax1.tick_params(axis="y", labelcolor="royalblue")

    # 응급관리요원 수 → 선 그래프로 표시 (오른쪽 Y축)
    ax2 = ax1.twinx()
    ax2.plot(years, emergency_staff, marker="o", color="red", linestyle="-", label="응급관리요원 수")
    ax2.set_ylabel("응급관리요원 수", color="red")
    ax2.set_ylim(0, global_y2_max)
    ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))
    ax2.tick_params(axis="y", labelcolor="red")

    # 제목 및 범례 추가
    plt.title(f"{region} 변화")
    fig.tight_layout()
    fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.85))

    # 차트를 Base64로 변환
    img = BytesIO()
    plt.savefig(img, format="png", bbox_inches="tight")
    img.seek(0)
    img_base64 = base64.b64encode(img.read()).decode("utf-8")
    plt.close()

    # Base64 저장
    region_chart_base64[region] = img_base64

# 대한민국 지도 생성 (Folium)
m = folium.Map(location = [36, 128], zoom_start = 7, tiles = "cartodb positron")

# 서비스 대상자 최소 및 최대값 가져오기
min_target = df_t[f"2024년_서비스이용자"].min()
max_target = df_t[f"2024년_서비스이용자"].max()

# 색상 스케일 설정
colormap = branca.colormap.LinearColormap(
    colors=["#FEE5D9", "#FC9272", "#DE2D26"],  # 밝은색 → 진한색 (적은 대상자 → 많은 대상자)
    vmin = min_target,
    vmax = max_target
)

# GeoJSON 파일 로드 및 Choropleth 추가
geojson_path = "project4/TL_SCCO_CTPRVN.json"  # GeoJSON 파일 경로
with open(geojson_path, encoding="utf-8") as f:
    geojson_data = json.load(f)

# 지역별 서비스 대상자 수 매핑
region_target_map = df_t.set_index("행정구역")[f"2024년_서비스이용자"].to_dict()

# 기존 속성을 완전히 덮어씌워 불필요한 정보 제거
for feature in geojson_data["features"]:
    region_name = feature["properties"]["CTP_KOR_NM"]

    # 차트 Base64 추가 (없으면 기본 메시지 출력)
    if region_name in region_chart_base64:
        chart_img = f'<img src="data:image/png;base64,{region_chart_base64[region_name]}" width="300">'
    else:
        chart_img = "<p>차트 데이터 없음</p>"

    # 기존 속성을 완전히 덮어씌우고, 필요한 값만 추가
    feature["properties"] = {
        "지역명": region_name,
        "서비스 이용자": f"{region_target_map.get(region_name, 'N/A'):,} 명",  # 1000단위 콤마 추가
        "차트": chart_img
    }

# 지도에 Choropleth(색상 맵) 추가
choropleth = folium.Choropleth(
    geo_data = geojson_data,
    name = "Choropleth",
    data = df_t,
    columns = ["행정구역", f"2024년_서비스이용자"],
    key_on = "feature.properties.지역명",
    fill_color = "YlOrRd",
    fill_opacity = 0.9,
    line_opacity = 0.4,
    legend_name = "서비스 이용자(명)",
    highlight = True
).add_to(m)

# 툴팁에 차트 포함 (불필요한 정보 제거)
folium.GeoJsonTooltip(
    fields = ["지역명", "서비스 이용자", "차트"],  # 불필요한 정보 제거 후 필요한 값만 유지
    aliases = ["지역명:", "서비스 이용자:", "변화 추이:"],
    localize = True,
    sticky = True,
    labels = True,
    style = "font-size: 12px; background-color: white; color: black;",
    escape = False  # HTML 렌더링 가능하게 설정
).add_to(choropleth.geojson)

# 지도 저장 및 표시
m.save("project4/서비스이용자_지도.html")
IFrame("project4/서비스이용자_지도.html", width=800, height=600)

In [13]:
plt.rc("font", family="Malgun Gothic")

# 연도 리스트
years = ["2020년", "2021년", "2022년", "2023년", "2024년"]

# "서비스 대상자(명)"을 숫자로 변환
for year in years:
    df_t[f"{year}_서비스이용자"] = df_t[f"{year}_서비스이용자"].astype(int)
    df_t[f"{year}_응급관리요원"] = df_t[f"{year}_응급관리요원"].astype(int)

# 지역별 차트 생성 (Base64 변환)
region_chart_base64 = {}

# 전체 데이터에서 최댓값 찾기 (Y축 범위 통일)
global_y_max = df_t[[f"{year}_서비스이용자" for year in years]].max().max() + 5000
global_y2_max = df_t[[f"{year}_응급관리요원" for year in years]].max().max() + 10

for _, row in df_t.iterrows():
    region = row["행정구역"]
    service_users = row[[f"{year}_서비스이용자" for year in years]].values
    emergency_staff = row[[f"{year}_응급관리요원" for year in years]].values

    fig, ax1 = plt.subplots(figsize=(5, 3))

    # 서비스 이용자 수 → 막대그래프로 표시 (왼쪽 Y축)
    ax1.bar(years, service_users, color="royalblue", alpha=0.7, label="서비스 이용자 수")
    ax1.set_ylabel("서비스 이용자 수", color="royalblue")
    ax1.set_ylim(0, global_y_max)
    ax1.yaxis.set_major_locator(ticker.MultipleLocator(5000))
    ax1.tick_params(axis="y", labelcolor="royalblue")

    # 응급관리요원 수 → 선 그래프로 표시 (오른쪽 Y축)
    ax2 = ax1.twinx()
    ax2.plot(years, emergency_staff, marker="o", color="red", linestyle="-", label="응급관리요원 수")
    ax2.set_ylabel("응급관리요원 수", color="red")
    ax2.set_ylim(0, global_y2_max)
    ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))
    ax2.tick_params(axis="y", labelcolor="red")

    # 제목 및 범례 추가
    plt.title(f"{region} 변화")
    fig.tight_layout()
    fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.85))

    # 차트를 Base64로 변환
    img = BytesIO()
    plt.savefig(img, format="png", bbox_inches="tight")
    img.seek(0)
    img_base64 = base64.b64encode(img.read()).decode("utf-8")
    plt.close()

    # Base64 저장
    region_chart_base64[region] = img_base64

# 대한민국 지도 생성 (Folium)
m = folium.Map(location = [36, 128], zoom_start = 7, tiles = "cartodb positron")

# 서비스 대상자 최소 및 최대값 가져오기
min_target = df_t[f"2024년_서비스이용자"].min()
max_target = df_t[f"2024년_서비스이용자"].max()

# 색상 스케일 설정
colormap = branca.colormap.LinearColormap(
    colors=["#FEE5D9", "#FC9272", "#DE2D26"],  # 밝은색 → 진한색 (적은 대상자 → 많은 대상자)
    vmin = min_target,
    vmax = max_target
)

# GeoJSON 파일 로드 및 Choropleth 추가
geojson_path = "project4/TL_SCCO_CTPRVN.json"  # GeoJSON 파일 경로
with open(geojson_path, encoding="utf-8") as f:
    geojson_data = json.load(f)

# 지역별 서비스 대상자 수 매핑
region_target_map = df_t.set_index("행정구역")[f"2024년_서비스이용자"].to_dict()

# 기존 속성을 완전히 덮어씌우고, "요원 1명당 담당 대상자 수(명)" 추가
for feature in geojson_data["features"]:
    region_name = feature["properties"]["CTP_KOR_NM"]

    # 데이터 매핑
    service_count = region_target_map.get(region_name, "N/A")
    staff_count = df_per.set_index("행정구역")["요원 1명당 담당 이용자 수"].to_dict().get(region_name, "N/A")

    # 🔹 소수점 제거 (int 변환)
    staff_count = int(staff_count) if staff_count != "N/A" else "N/A"

    # 차트 Base64 추가 (없으면 기본 메시지 출력)
    chart_img = f'<img src="data:image/png;base64,{region_chart_base64.get(region_name, "")}" width="300">' if region_name in region_chart_base64 else "<p>차트 데이터 없음</p>"

    # 속성 업데이트
    feature["properties"] = {
        "지역명": region_name,
        "서비스 대상자": f"{service_count:,} 명" if service_count != "N/A" else "N/A",
        "요원 1명당 담당 이용자 수": f"{staff_count:,} 명" if staff_count != "N/A" else "N/A",
        "차트": chart_img
    }

# 지도에 Choropleth(색상 맵) 추가
choropleth = folium.Choropleth(
    geo_data = geojson_data,
    name = "Choropleth",
    data = df_t,
    columns = ["행정구역", f"2024년_서비스이용자"],
    key_on = "feature.properties.지역명",
    fill_color = "YlOrRd",
    fill_opacity = 0.9,
    line_opacity = 0.4,
    legend_name = "서비스 이용자(명)",
    highlight = True
).add_to(m)

# 🔹 툴팁 업데이트
folium.GeoJsonTooltip(
    fields=["지역명", "서비스 대상자", "요원 1명당 담당 이용자 수", "차트"],
    aliases=["지역명:", "서비스 대상자:", "요원 1명당 담당 이용자 수:", "변화 추이:"],
    localize=True,
    sticky=True,
    labels=True,
    style="font-size: 12px; background-color: white; color: black;",
    escape=False  # 🔹 HTML 적용 가능하도록 설정
).add_to(choropleth.geojson)

# 지도 저장 및 표시
m.save("project4/인원현황.html")
IFrame("project4/인원현황.html", width=800, height=600)

### 최종 결과물

In [15]:
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>응급안전안심서비스_종합지도</title>
    <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
    <style>
        .radio-container {{
            position: absolute;
            top: 10px;
            left: 10px;
            background: rgba(255, 255, 255, 0.9);
            padding: 10px;
            border-radius: 8px;
            z-index: 1000;
            font-size: 14px;
            box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.2);
            display: flex;
            flex-direction: column;
            gap: 5px;
        }}
        .map-container {{
            display: none;
            width: 100%;
            height: 100vh;
        }}
    </style>
</head>
<body>
    <div class="radio-container">
        <label><input type="radio" name="mapSelect" value="care" checked> 노인돌봄서비스기관</label>
        <label><input type="radio" name="mapSelect" value="safety"> 응급안전안심서비스</label>
        <label><input type="radio" name="mapSelect" value="safety_care"> 응급안전안심 방문요양서비스</label>
        <label><input type="radio" name="mapSelect" value="personnel"> 인원 현황</label>
    </div>

    <iframe id="care" class="map-container" src="https://storage.googleapis.com/sungnam_charger_map/%EB%85%B8%EC%9D%B8%EB%8F%8C%EB%B4%84%EC%84%9C%EB%B9%84%EC%8A%A4%EA%B8%B0%EA%B4%80.html" style="display:block;"></iframe>
    <iframe id="safety" class="map-container" src="https://storage.googleapis.com/sungnam_charger_map/%EC%9D%91%EA%B8%89%EC%95%88%EC%A0%84%EC%95%88%EC%8B%AC%EC%84%9C%EB%B9%84%EC%8A%A4.html"></iframe>
    <iframe id="safety_care" class="map-container" src="https://storage.googleapis.com/sungnam_charger_map/%EC%9D%91%EA%B8%89%EC%95%88%EC%A0%84%EC%95%88%EC%8B%AC_%EB%B0%A9%EB%AC%B8%EC%9A%94%EC%96%91%EC%84%9C%EB%B9%84%EC%8A%A4.html"></iframe>
    <iframe id="personnel" class="map-container" src="https://storage.googleapis.com/sungnam_charger_map/%EC%9D%B8%EC%9B%90%ED%98%84%ED%99%A9.html"></iframe>

    <script>
        $(document).ready(function() {{
            $('input[name="mapSelect"]').change(function() {{
                $('.map-container').hide();
                $('#' + $(this).val()).show();
            }});
        }});
    </script>
</body>
</html>
"""

# HTML 파일 저장
with open("응급안전안심서비스_종합지도.html", "w", encoding="utf-8") as f:
    f.write(html_content)

# Jupyter Notebook에서 HTML 파일 실행
webbrowser.open("응급안전안심서비스_종합지도.html") 


True